In [1]:
import numpy as np
import os
import pandas as pd
from sklearn import tree
from sklearn import preprocessing

In [2]:
# Read US Census construction dataset

os.chdir( '/Users/healey/Downloads' )

# Drop the first row, it's an "explanation" row and doesn't contain data,
# but since it has strings it forces every column's values to also be
# strings, which is NOT what we want

df = pd.read_csv( 'construction-census.csv', sep=',', skiprows=[ 1 ] )

In [3]:
col = df[ 'PAYANN' ]    # Get PAYANN column

# loc() is used for label-based indexing

row = df.loc[ 3 ]    # Get row with label of '3'
row = df.loc[ 2: 4 ]  # Get rows with labels of '2', '3', and '4'
row = df.loc[ [ 0, 2, 4 ] ]    # Get rows with labels of '0', '2', and '4'

# For columns, specify : as first argument to get all rows

col = df.loc[ :, 'PAYANN' ]    # Get column with label of 'PAYANN'
col = df.loc[ :, 'ESTAB': 'EMP' ]  # Get columns from 'ESTAB' to 'EMP'
col = df.loc[ :, [ 'EMP', 'PAYANN', 'BENVOL'] ]    # Get columns 'EMP', 'PAYANN', 'BENVOL'

# iloc() is used for absolute (integer) indexing

row = df.iloc[ 2 ]    # Get 3rd row (remember, indexing is 0-based)
row = df.iloc[ 0: 3 ]    # Get 1st thru 3rd rows
row = df.iloc[ [ 0, 2, 4 ] ]    # Get 1st, 3rd, and 5th rows

col = df.iloc[ :, 2 ]    # Get 3rd column
col = df.iloc[ :, 2: 4 ]    # Get 3rd and 4th column
col = df.iloc[ :, [0, 2, 4 ] ]    # Get 1st, 3rd, and 5th columns

In [4]:
# Get list of column names

col_nm = list( df )
print col_nm

['GEO.id', 'GEO.id2', 'GEO.display-label', 'GEO.annotation.id', 'NAICS.id', 'NAICS.display-label', 'NAICS.annotation.id', 'YEAR.id', 'ESTAB', 'ESTAB_S', 'EMP', 'EMP_S', 'PAYANN', 'PAYANN_S', 'BENEFIT', 'BENEFIT_S', 'BENLGL', 'BENVOL', 'EMPSMCD', 'EMPSMCD_S', 'EMPQ1CW', 'EMPQ2CW', 'EMPQ3CW', 'EMPQ4CW', 'EMPSMOD', 'EMPSMOD_S', 'EMPQ1OC', 'EMPQ2OC', 'EMPQ3OC', 'EMPQ4OC', 'PAYANOC', 'PAYANOC_S', 'HOURS', 'HOURS_S', 'PAYANCW', 'PAYANCW_S', 'PAYQTR1', 'PAYQTR1_S', 'CSTCMT', 'CSTMPRT', 'CSTMPRT_S', 'CSTSCNT', 'CSTSCNT_S', 'CSTEFT', 'CSTEFT_S', 'CSTFUGT', 'CSTFUNG', 'CSTFUON', 'CSTFUOF', 'CSTFUOT', 'CSTELEC', 'CSTELEC_S', 'RCPTOT', 'RCPTOT_S', 'RCPCWRK', 'RCPCWRK_S', 'RCPCGDL', 'FEDDL', 'SLDL', 'PRIDL', 'RCPOTH', 'RCPOTH_S', 'RCPCCNDL', 'RCPNCW', 'RCPNCW_S', 'VALADD', 'VALADD_S', 'INVMATB', 'INVMATB_S', 'INVMATE', 'INVMATE_S', 'ESTCIN', 'ESTCIN_S', 'RCPCIN', 'RCPCIN_S', 'ESTCNO', 'ESTCNO_S', 'RCPCNO', 'RCPCNO_S', 'ESTCNR', 'ESTCNR_S', 'RCPCNR', 'RCPCNR_S', 'ASTBGN', 'CEXTOT', 'CEXTOT_S', 'ASRE

In [5]:
# Get row and column counts, and number of observations

shp = df.shape
row = shp[ 0 ]
col = shp[ 1 ]
n = row * col
print 'Rows:', row, ' Columns:', col, ' Observations:', n

Rows: 1575  Columns: 108  Observations: 170100


In [6]:
# Count number of missing (NaN) values

nan_array = df.isnull().values
nan_list = nan_array.tolist()

nan_n = 0
for i in range( 0, len( nan_list ) ):
    for j in range( 0, len( nan_list[ i ] ) ):
        if nan_list[ i ][ j ] == True:
            nan_n = nan_n + 1

print nan_n

25905


In [7]:
# Or, comperss the 2D list into a 1D list, count number of True values

nan_list = [ val for sublist in nan_list for val in sublist ]
nan_n = nan_list.count( True )
print nan_n

25905


In [8]:
# Or, rely on the fact that True=1, False=0 and sum the True/False value list

nan_n = df.isnull().values.sum()
print nan_n

25905


In [9]:
# To copy a DataFrame, you cannot simply equate them (df_cp = df), this creates a
# "reference" from df_cp and df to the same DataFrame; instead, use copy()

df_cp = df.copy()
col_nm = list( df_cp )

# Remove any column that's entirely NaN

for nm in col_nm:
    nan_n = df_cp[ nm ].isnull().values.sum()
    if nan_n > 0:
        print nm, 'has', nan_n, 'NaN values;',
        
    # Remove column if it's completely empty
    
    if nan_n == row:
        print 'Removing empty column', nm
        del df_cp[ nm ]

GEO.annotation.id has 1575 NaN values; Removing empty column GEO.annotation.id
NAICS.annotation.id has 1575 NaN values; Removing empty column NAICS.annotation.id
EMP has 38 NaN values; EMP_S has 38 NaN values; PAYANN has 57 NaN values; PAYANN_S has 57 NaN values; BENEFIT has 61 NaN values; BENEFIT_S has 61 NaN values; BENLGL has 91 NaN values; BENVOL has 89 NaN values; EMPSMCD has 78 NaN values; EMPSMCD_S has 81 NaN values; EMPQ1CW has 88 NaN values; EMPQ2CW has 71 NaN values; EMPQ3CW has 68 NaN values; EMPQ4CW has 68 NaN values; EMPSMOD has 78 NaN values; EMPSMOD_S has 79 NaN values; EMPQ1OC has 60 NaN values; EMPQ2OC has 62 NaN values; EMPQ3OC has 72 NaN values; EMPQ4OC has 70 NaN values; PAYANOC has 103 NaN values; PAYANOC_S has 104 NaN values; HOURS has 62 NaN values; HOURS_S has 65 NaN values; PAYANCW has 102 NaN values; PAYANCW_S has 105 NaN values; PAYQTR1 has 63 NaN values; PAYQTR1_S has 63 NaN values; CSTCMT has 165 NaN values; CSTMPRT has 122 NaN values; CSTMPRT_S has 122 NaN

In [10]:
# NaN values can be imputed in many different ways

# fillna() fills NaN values with a fixed constant

df_imp = df.copy()
df_imp = df_imp.fillna( 0 )    # Fill all NaN in DataFrame with 0

# fillna() can also be used with method='pad' to fill NaN with preceding
# non-NaN value, or with method='backfill' to fill NaN with succeeding
# non-NaN value; note that this will not fix a column of all NaN

df_imp = df.copy()
df_imp = df_imp.fillna( method='pad' )
df_imp = df_imp.fillna( method='backfill' )

# interpolate() fills values by interpolating over one or more NaN, different
# methods are available like linear, nearest, spline, etc.

# Note that interpolate() will throw an error if a column is all NaN

df_imp = df.copy()
df_imp[ 'BENVOL' ] = df_imp[ 'BENVOL' ].interpolate()

print df[ 'BENVOL' ]
print df_imp[ 'BENVOL' ]

0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560      NaN
1561     1891
1562      100
1563     1530
1564    10625
1565     7999
1566      593
1567     1654
1568      544
1569      NaN
1570      330
1571      303
1572      NaN
1573     6173
1574     1234
Name: BENVOL, Length: 1575, dtype: float64
0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560     1150.5
1561     1891.0
1562      100.0
1563     1530.0
1564    10625.0
1565     7999.0
1566      593.0
1567     1654.0
1568      544.0
1569      437.0
1570      330.0
1571      303.0
1572     3238.0
1573     6173.0
1574     1234.0
Name: BENVOL, Length: 1575, dtype: float64


In [11]:
# scikit-learn can also impute NaN with its Imputer class, replacing NaN with
# mean, median, or most_frequent value for a column/axis or entire DataFrame

# Imputer cannot handle strings, however, so we must first convert any columns
# with string values to numeric representations using Label Encoder

df_imp = df_cp.copy()
col_nm = list( df_cp )

le = { }    # Label encoders, one per string column, key is column label

for nm in col_nm:
    
    # Try to check for any string value, this will throw an error if the column
    # is all-numeric, so catch that with except and do nothing (pass)
    
    try:
        if True in df_imp[ nm ].str.contains( '.+' ).values:
            print 'Column', nm, 'contains string(s), label encoding...'
            
            le[ nm ] = preprocessing.LabelEncoder()
            le[ nm ].fit( df_imp[ nm ] )
            df_imp[ nm ] = pd.Series( le[ nm ].transform( df_imp[ nm ] ) )
    except:
        pass

imp = preprocessing.Imputer( strategy='median' )
imp.fit( df_imp )

col_nm = list( df_imp )
df_imp = pd.DataFrame( imp.transform( df_imp ), columns=col_nm )

print df[ 'BENVOL' ]
print df_imp[ 'BENVOL' ]

Column GEO.id contains string(s), label encoding...
Column GEO.display-label contains string(s), label encoding...
Column NAICS.display-label contains string(s), label encoding...
0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560      NaN
1561     1891
1562      100
1563     1530
1564    10625
1565     7999
1566      593
1567     1654
1568      544
1569      NaN
1570      330
1571      303
1572      NaN
1573     6173
1574     1234
Name: BENVOL, Length: 1575, dtype: float64
0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560     6184
1561     1891
1562      100
1563     1530
1564    10625
1565     7999
1566      593
1567     1654
1568      544
1569     6184
1570      330
1571      303
1572     6184
1573     6173
157

In [12]:
# We can also try to impute NaN using classification, e.g. a tree classifier; let's
# try to impute NaN in the BENVOL column

# Start by creating a training set whose rows have no NaN

train = pd.DataFrame()

for i, row in df_cp.iterrows():
    if row.isnull().values.sum() == 0:
        train = train.append( row )
        
# LabelEncode any columns with strings, because the classifier doesn't
# handle strings

col_nm = list( train )

for nm in col_nm:
    try:
        if True in train[ nm ].str.contains( '.+' ).values:
            print 'Column', nm, 'contains string(s), label encoding...'
            
            le = preprocessing.LabelEncoder()
            le.fit( train[ nm ] )
            
            # Because we've removed rows w/NaN from train when we created
            # it, its indices aren't contiguous 0,1,2,... they're based on
            # the rows we kept 1,2,8,12,19... so when we build the label
            # encoded series, we MUST match the same index values
            
            train[ nm ] = pd.Series( le.transform( train[ nm ] ), index=train[ nm ].index )
    except:
        pass

# Now, grab the BENVOL column as a "target" and remove it from train, so we can
# train on all columns except BENVOL to estimate BENVOL

tg = train[ 'BENVOL' ]
del train[ 'BENVOL' ]

# Discretize the continuous BENVOL values into bins

tg = pd.cut( tg, 100 )

# Create and fit a sci-learn tree classifier

clf = tree.DecisionTreeClassifier()
clf.fit( train, tg )

Column GEO.display-label contains string(s), label encoding...
Column GEO.id contains string(s), label encoding...
Column NAICS.display-label contains string(s), label encoding...


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [13]:
# Now use the tree classifier to impute NaN in the original BENVOL column

tg = df[ 'BENVOL' ].copy()

# Use df_imp, which has all NaN imputed, because the classifier can't
# handle NaN

df_known = df_imp.copy()
del df_known[ 'BENVOL' ]

# Now walk through all BENVOL entries, use the classifier to impute any NaN

for i in range( 0, len( tg ) ):
    if np.isnan( tg.iloc[ i ] ):
        
        # Reshape an n x 1 column into a 1 x n vector, since the classifier wants
        # known values in this format
        
        vec = df_known.iloc[ i ].reshape( 1, -1 )
        rng = clf.predict( vec )[ 0 ]
        
        # Predictions are a string '(lo, hi]', so we parse this to get the
        # median value between lo and hi and use that as our prediction
        
        rng = rng[ 1: -1 ]
        rng = rng.split( ', ' )
        val = float( rng[ 0 ] ) + float( rng[ 1 ] ) / 2.0

        tg.iloc[ i ] = val

print df[ 'BENVOL' ]
print tg

0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560      NaN
1561     1891
1562      100
1563     1530
1564    10625
1565     7999
1566      593
1567     1654
1568      544
1569      NaN
1570      330
1571      303
1572      NaN
1573     6173
1574     1234
Name: BENVOL, Length: 1575, dtype: float64
0      2078
1      1128
2      4343
3      3875
4     20944
5     50004
6      8914
7      6395
8     10538
9       326
10    23572
11     2652
12     2996
13     1671
14      190
...
1560     5694.128
1561     1891.000
1562      100.000
1563     1530.000
1564    10625.000
1565     7999.000
1566      593.000
1567     1654.000
1568      544.000
1569     5694.128
1570      330.000
1571      303.000
1572     5694.128
1573     6173.000
1574     1234.000
Name: BENVOL, Length: 1575, dtype: float64


####### 